# 🚀 生产级学术术语提取系统 (Production-Grade Terminology Extraction)

**本代码旨在解决传统 NLP 提取中的三大痛点：**
1.  **噪音控制：** 自动剔除 `et al`, `Figure 1`, `DOI` 等非语义内容。
2.  **语法精准：** 使用 POS Pattern (词性模式) 替代粗糙的名词短语提取，只保留 `Adj+Noun` 或 `Noun+Noun`。
3.  **权重校准：** 修复 TF-IDF 计算逻辑，确保长尾专业术语能获得高分。

---

In [64]:
# === Cell 1: 环境配置与依赖检查 ===
import sys
import os

# 1. 定义依赖库
packages = [
    "pymupdf",       # PDF 解析
    "spacy",         # 高级 NLP
    "scikit-learn",  # TF-IDF 算法
    "pandas",        # 数据表格
    "openpyxl",      # Excel 导出
    "requests",      # 下载测试文件
    "nltk"           # 辅助停用词库
]

print("🛠️ 正在检查环境...")
for pkg in packages:
    try:
        __import__(pkg)
    except ImportError:
        print(f"  ⬇️ 正在安装缺失库: {pkg}...")
        !{sys.executable} -m pip install {pkg} -q

# 2. 下载 spaCy 模型与 NLTK 数据
import spacy
import nltk

try:
    nlp = spacy.load("en_core_web_md")
except OSError:
    print("  ⬇️ 下载 spaCy 英文模型...")
    !{sys.executable} -m spacy download en_core_web_sm
    nlp = spacy.load("en_core_web_md")

nltk.download('stopwords', quiet=True)
print("✅ 所有系统依赖已就绪！")

🛠️ 正在检查环境...
  ⬇️ 正在安装缺失库: scikit-learn...
✅ 所有系统依赖已就绪！


In [65]:
# === Cell 2: 文本清洗与 PDF 解析模块 ===
import fitz  # PyMuPDF
import re
import requests

class DocumentProcessor:
    @staticmethod
    def normalize_text(text):
        """
        【核心清洗函数】
        必须确保提取阶段和 TF-IDF 阶段使用完全相同的清洗逻辑，
        否则会导致匹配失败（分数归零）。
        """
        if not text: return ""
        
        # 1. 修复 PDF 换行连字符 (ex- ample -> example)
        text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text)
        
        # 2. 移除引用标记 [1], [12-14]
        text = re.sub(r'\[\d+(?:-\d+)?\]', '', text)
        
        # 3. 移除特殊字符，只保留字母和空格
        # 这一步非常重要，它去除了 "30%" 中的 %，防止 "30%" 被识别为名词
        text = re.sub(r'[^a-zA-Z\s]', ' ', text)
        
        # 4. 压缩多余空格并转小写
        text = re.sub(r'\s+', ' ', text).strip().lower()
        return text

    @staticmethod
    def parse_pdf(pdf_path):
        """解析 PDF，返回 (全文, 页面列表)"""
        if not os.path.exists(pdf_path):
            raise FileNotFoundError(f"❌ 文件未找到: {pdf_path}")
            
        doc = fitz.open(pdf_path)
        full_text = []
        pages_corpus = []
        
        print(f"📖 正在解析 {os.path.basename(pdf_path)} (共 {len(doc)} 页)...")
        for page in doc:
            text = page.get_text()
            # 简单过滤掉太短的页面（通常是图片或只有页码）
            if len(text) > 100:
                # 注意：这里我们保留原始文本结构给 TF-IDF，
                # 但在内部计算时会调用 normalize_text
                pages_corpus.append(text) 
                full_text.append(text)
                
        return " ".join(full_text), pages_corpus

    @staticmethod
    def download_sample(url, filename="sample.pdf"):
        if not os.path.exists(filename):
            print(f"⬇️ 正在下载测试文献: {url}...")
            r = requests.get(url)
            with open(filename, 'wb') as f:
                f.write(r.content)
        return filename

# 准备测试数据
pdf_url = "https://arxiv.org/pdf/1706.03762.pdf" # Transformer Paper
local_pdf = DocumentProcessor.download_sample(pdf_url, "paper3.pdf")
raw_text, raw_pages = DocumentProcessor.parse_pdf(local_pdf)
print("✅ 文本预处理模块就绪。")

📖 正在解析 paper3.pdf (共 15 页)...
✅ 文本预处理模块就绪。


In [66]:
# === Cell 3: 生产级核心算法引擎 (Sniper Edition) ===
import spacy
from spacy.matcher import Matcher
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
import pandas as pd

class TerminologyExtractor:
    def __init__(self, nlp_model):
        self.nlp = nlp_model
        self.matcher = Matcher(self.nlp.vocab)
        
        # --- 1. 严格语法模式 ---
        # 仅保留最有信息量的结构
        self.matcher.add("TERM", [
            [{"POS": "ADJ"}, {"POS": "NOUN"}],              # e.g., clinical trial
            [{"POS": "NOUN"}, {"POS": "NOUN"}],              # e.g., opioid misuse
            [{"POS": "ADJ"}, {"POS": "NOUN"}, {"POS": "NOUN"}], # e.g., chronic heart failure
            [{"POS": "NOUN"}, {"POS": "NOUN"}, {"POS": "NOUN"}]  # e.g., body mass index
        ])
        
        # --- 2. 语义狙击黑名单 (Targeted Blacklist) ---
        self.blacklist_tokens = {
            # ==============================================
            # 1. 元数据与引用噪声 (顶级优先级，必过滤)
            # ==============================================
            "et", "al", "et al", "figure", "fig", "table", "tbl", "doi", "http", "https", 
            "www", "url", "pdf", "html", "xml", "volume", "vol", "issue", "iss", "page", 
            "pp", "p", "author", "authors", "press", "publisher", "copyright", "copyrighted", 
            "reserved", "all rights reserved", "permission", "license", "licence", "creative commons",
            "cc", "publication", "published", "print", "online", "abstract", "keywords", "reference",
            "references", "citation", "citations", "bibliography", "ref", "refs",
        
            # ==============================================
            # 2. 图表与视觉类噪声 (论文图表/可视化相关，无翻译价值)
            # ==============================================
            "line", "solid", "dashed", "dotted", "dash-dot", "circle", "circles", 
            "triangle", "triangles", "square", "squares", "diamond", "diamonds", 
            "axis", "axes", "plot", "plots", "curve", "curves", "graph", "graphs", 
            "color", "colour", "red", "blue", "green", "yellow", "black", "white", 
            "grey", "gray", "purple", "orange", "pink", "brown", "shown", "shown in", 
            "represented", "representation", "illustrated", "illustration", "bar", 
            "bars", "histogram", "heatmap", "scatter", "boxplot", "error bar", "legend",
        
            # ==============================================
            # 3. 学术通用废话 (无专业价值的学术框架词)
            # ==============================================
            "study", "studies", "data", "dataset", "datasets", "result", "results", 
            "analysis", "analyses", "method", "methods", "methodology", "conclusion", 
            "conclusions", "discussion", "introduction", "background", "objective", 
            "objectives", "aim", "aims", "purpose", "purposes", "finding", "findings", 
            "observation", "observations", "note", "notes", "remark", "remarks", 
            "section", "sections", "part", "parts", "chapter", "chapters", "article", 
            "paper", "manuscript", "text", "paragraph", "para", "supplementary", "suppl",
            "appendix", "appendices", "supplemental", "s1", "s2", "extended data",
        
            # ==============================================
            # 4. 时间/统计/计量噪声 (通用时间/统计词，无专业翻译价值)
            # ==============================================
            "time", "times", "month", "months", "week", "weeks", "year", "years", 
            "day", "days", "hour", "hours", "minute", "minutes", "second", "seconds",
            "baseline", "follow", "follow-up", "followup", "period", "periods", 
            "duration", "interval", "intervals", "total", "average", "avg", "mean", 
            "median", "mode", "range", "number", "numbers", "score", "scores", 
            "rate", "rates", "ratio", "ratios", "level", "levels", "value", "values", 
            "difference", "differences", "comparison", "comparisons", "sample", 
            "samples", "size", "sizes", "frequency", "frequencies", "percentage", 
            "percent", "%", "proportion", "proportions", "count", "counts",
        
            # ==============================================
            # 5. 研究对象/分组噪声 (通用实验设计词)
            # ==============================================
            "participant", "participants", "subject", "subjects", "patient", "patients",
            "population", "populations", "cohort", "cohorts", "case", "cases", 
            "control", "controls", "group", "groups", "arm", "arms", "trial", "trials",
            "arm", "arms", "type", "types", "category", "categories", "class", "classes",
        
            # ==============================================
            # 6. 统计方法论噪声 (通用统计/建模词，无领域价值)
            # ==============================================
            "model", "models", "regression", "logistic", "linear", "mixed model", 
            "random", "randomized", "randomization", "intercept", "intercepts", 
            "variable", "variables", "factor", "factors", "effect", "effects", 
            "outcome", "outcomes", "measure", "measures", "metric", "metrics", 
            "test", "tests", "anova", "t-test", "chi-square", "chi2", "p-value", 
            "p value", "significance", "significant", "ns", "non-significant", 
            "confidence interval", "ci", "or", "odds ratio", "rr", "risk ratio", 
            "hr", "hazard ratio", "adjusted", "unadjusted", "estimate", "estimates",
        
            # ==============================================
            # 7. 期刊/出版专属噪声 (期刊名/出版方，无通用翻译价值)
            # ==============================================
            "nature", "health", "science", "cell", "nejm", "lancet", "bmj", "jama", 
            "plos one", "elsevier", "springer", "wiley", "taylor & francis", "sage", 
            "oxford university press", "cambridge university press",
        
            # ==============================================
            # 8. 项目/专属名词噪声 (项目名/机构名，无通用翻译价值)
            # ==============================================
            "playsmart", "gameplay", "play smart", "videogame", "video game", "videogames",
        
            # ==============================================
            # 9. 缩写/残词/OCR噪声 (OCR识别错误/不完整缩写)
            # ==============================================
            "subst", "abus", "prev", "treat", "dept", "univ", "inst", "conf", 
            "proc", "suppl", "nat", "int", "ext", "ci", "or", "rr", "hr", "vs", 
            "etc", "i.e", "e.g", "eg", "ie", "viz", "cf", "et seq", "seq", "ed", 
            "eds", "edn", "rev", "ed rev", "trans", "tr", "vols", "no", "nos", 
            "figs", "tabs", "suppl", "spp", "ms", "msn", "phd", "md", "dr", "prof",
        
            # ==============================================
            # 10. 通用废话修饰词 (无专业价值的形容词/名词)
            # ==============================================
            "great", "good", "bad", "high", "low", "big", "small", "large", 
            "little", "severe", "mild", "moderate", "major", "minor", "primary", 
            "secondary", "tertiary", "main", "chief", "key", "central", "core", 
            "total", "overall", "general", "specific", "particular", "various", 
            "several", "different", "similar", "same", "other", "another", "new", 
            "old", "young", "oldest", "youngest", "positive", "negative", "neutral",
            "common", "rare", "uncommon", "frequent", "infrequent"
        }
        # --- 3. 补充缺失的 bad_starters 属性 (解决 AttributeError 核心) ---
        self.bad_starters = {
            # 无效形容词前缀：以这些词开头的术语直接过滤
            "great", "good", "bad", "high", "low", "big", "small", "large",
            "significant", "total", "mean", "average", "various", "several",
            "different", "similar", "other", "new", "old", "positive", "negative",
            "primary", "secondary", "main", "major", "minor", "severe", "mild",
            "moderate", "general", "specific", "common", "rare", "frequent",
            "infrequent", "little", "chief", "key", "central", "core", "overall",
            "particular", "same", "another", "young", "oldest", "youngest",
            "neutral", "uncommon"
        }
        self.stop_words = set(stopwords.words('english'))

    @staticmethod
    def robust_clean(text):
        """深度清洗：保留句号作为语义阻断"""
        if not text: return ""
        
        # 1. 将换行连字符修复 (ex- ample -> example)
        text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text)
        
        # 2. 将非句号的标点转为空格 (保留 . 用于后续分句，但 spacy 不需要显式分句)
        # 这里策略调整：直接把所有非字母转为空格，依靠黑名单解决跨句粘连
        text = re.sub(r'[^a-zA-Z\s]', ' ', text)
        
        # 3. 强力去重 (months months)
        text = re.sub(r'\b(\w+)\s+\1\b', r'\1', text, flags=re.IGNORECASE)
        
        # 4. 压缩空格
        return re.sub(r'\s+', ' ', text).strip().lower()

    def is_valid_term(self, term):
        """语义过滤器"""
        words = term.split()
        
        # A. 长度硬限制
        if len(term) < 4 or len(words) > 3: return False
        
        # B. 黑名单检查 (只要包含任何一个黑名单词，直接丢弃)
        # 这会杀掉 "nature health" (含 nature), "baseline weeks" (含 weeks/baseline)
        if any(w in self.blacklist_tokens for w in words): return False
        
        # C. 形容词前缀检查
        if words[0] in self.bad_starters: return False
        
        # D. 停用词边界
        if words[0] in self.stop_words or words[-1] in self.stop_words: return False
        
        # E. 垃圾词检测 (无元音、乱码)
        for w in words:
            if len(w) < 2: return False
            if not re.search(r'[aeiouy]', w): return False # 必须包含元音
            
        return True

    def extract_candidates(self, text):
        clean_text = self.robust_clean(text)
        doc = self.nlp(clean_text[:1500000]) 
        matches = self.matcher(doc)
        
        candidates = set()
        for _, start, end in matches:
            span = doc[start:end]
            term = span.text.strip()
            if self.is_valid_term(term):
                candidates.add(term)
        return list(candidates)

    def compute_tfidf(self, pages_list, vocabulary):
        if not vocabulary: return {}
        print("📊 正在计算 TF-IDF 矩阵...")
        vectorizer = TfidfVectorizer(
            vocabulary=vocabulary,
            preprocessor=self.robust_clean,
            ngram_range=(1, 3),
            norm='l2'
        )
        try:
            X = vectorizer.fit_transform(pages_list)
            scores = X.sum(axis=0).A1
            return dict(zip(vectorizer.get_feature_names_out(), scores))
        except Exception as e:
            print(f"⚠️ TF-IDF Error: {e}")
            return {}

# 初始化
extractor = TerminologyExtractor(nlp)
print("✅ 核心算法引擎已构建 (Sniper Edition: 去除期刊名与方法论噪音)")

✅ 核心算法引擎已构建 (Sniper Edition: 去除期刊名与方法论噪音)


In [67]:
# === Cell 3: 生产级核心算法引擎 (Audit Edition: 带审计日志) ===
import spacy
from spacy.matcher import Matcher
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import re
import pandas as pd
from collections import Counter

class TerminologyExtractor:
    def __init__(self, nlp_model):
        self.nlp = nlp_model
        self.matcher = Matcher(self.nlp.vocab)
        self.rejection_log = [] # <--- 新增：用于存储被枪毙的词
        
        # --- 1. 严格语法模式 ---
        self.matcher.add("TERM", [
            [{"POS": "ADJ"}, {"POS": "NOUN"}],
            [{"POS": "NOUN"}, {"POS": "NOUN"}],
            [{"POS": "ADJ"}, {"POS": "NOUN"}, {"POS": "NOUN"}],
            [{"POS": "NOUN"}, {"POS": "NOUN"}, {"POS": "NOUN"}]
        ])
        
        # --- 2. 语义狙击黑名单 ---
       # --- 2. 语义狙击黑名单 (Targeted Blacklist) ---
        self.blacklist_tokens = {
            # ==============================================
            # 1. 元数据与引用噪声 (顶级优先级，必过滤)
            # ==============================================
            "et", "al", "et al", "figure", "fig", "table", "tbl", "doi", "http", "https", 
            "www", "url", "pdf", "html", "xml", "volume", "vol", "issue", "iss", "page", 
            "pp", "p", "author", "authors", "press", "publisher", "copyright", "copyrighted", 
            "reserved", "all rights reserved", "permission", "license", "licence", "creative commons",
            "cc", "publication", "published", "print", "online", "abstract", "keywords", "reference",
            "references", "citation", "citations", "bibliography", "ref", "refs",
        
            # ==============================================
            # 2. 图表与视觉类噪声 (论文图表/可视化相关，无翻译价值)
            # ==============================================
            "line", "solid", "dashed", "dotted", "dash-dot", "circle", "circles", 
            "triangle", "triangles", "square", "squares", "diamond", "diamonds", 
            "axis", "axes", "plot", "plots", "curve", "curves", "graph", "graphs", 
            "color", "colour", "red", "blue", "green", "yellow", "black", "white", 
            "grey", "gray", "purple", "orange", "pink", "brown", "shown", "shown in", 
            "represented", "representation", "illustrated", "illustration", "bar", 
            "bars", "histogram", "heatmap", "scatter", "boxplot", "error bar", "legend",
        
            # ==============================================
            # 3. 学术通用废话 (无专业价值的学术框架词)
            # ==============================================
            "study", "studies", "data", "dataset", "datasets", "result", "results", 
            "analysis", "analyses", "method", "methods", "methodology", "conclusion", 
            "conclusions", "discussion", "introduction", "background", "objective", 
            "objectives", "aim", "aims", "purpose", "purposes", "finding", "findings", 
            "observation", "observations", "note", "notes", "remark", "remarks", 
            "section", "sections", "part", "parts", "chapter", "chapters", "article", 
            "paper", "manuscript", "text", "paragraph", "para", "supplementary", "suppl",
            "appendix", "appendices", "supplemental", "s1", "s2", "extended data",
        
            # ==============================================
            # 4. 时间/统计/计量噪声 (通用时间/统计词，无专业翻译价值)
            # ==============================================
            "time", "times", "month", "months", "week", "weeks", "year", "years", 
            "day", "days", "hour", "hours", "minute", "minutes", "second", "seconds",
            "baseline", "follow", "follow-up", "followup", "period", "periods", 
            "duration", "interval", "intervals", "total", "average", "avg", "mean", 
            "median", "mode", "range", "number", "numbers", "score", "scores", 
            "rate", "rates", "ratio", "ratios", "level", "levels", "value", "values", 
            "difference", "differences", "comparison", "comparisons", "sample", 
            "samples", "size", "sizes", "frequency", "frequencies", "percentage", 
            "percent", "%", "proportion", "proportions", "count", "counts",
        
            # ==============================================
            # 5. 研究对象/分组噪声 (通用实验设计词)
            # ==============================================
            "participant", "participants", "subject", "subjects", "patient", "patients",
            "population", "populations", "cohort", "cohorts", "case", "cases", 
            "control", "controls", "group", "groups", "arm", "arms", "trial", "trials",
            "arm", "arms", "type", "types", "category", "categories", "class", "classes",
        
            # ==============================================
            # 6. 统计方法论噪声 (通用统计/建模词，无领域价值)
            # ==============================================
            "model", "models", "regression", "logistic", "linear", "mixed model", 
            "random", "randomized", "randomization", "intercept", "intercepts", 
            "variable", "variables", "factor", "factors", "effect", "effects", 
            "outcome", "outcomes", "measure", "measures", "metric", "metrics", 
            "test", "tests", "anova", "t-test", "chi-square", "chi2", "p-value", 
            "p value", "significance", "significant", "ns", "non-significant", 
            "confidence interval", "ci", "or", "odds ratio", "rr", "risk ratio", 
            "hr", "hazard ratio", "adjusted", "unadjusted", "estimate", "estimates",
        
            # ==============================================
            # 7. 期刊/出版专属噪声 (期刊名/出版方，无通用翻译价值)
            # ==============================================
            "nature", "health", "science", "cell", "nejm", "lancet", "bmj", "jama", 
            "plos one", "elsevier", "springer", "wiley", "taylor & francis", "sage", 
            "oxford university press", "cambridge university press",
        
            # ==============================================
            # 8. 项目/专属名词噪声 (项目名/机构名，无通用翻译价值)
            # ==============================================
            "playsmart", "gameplay", "play smart", "videogame", "video game", "videogames",
        
            # ==============================================
            # 9. 缩写/残词/OCR噪声 (OCR识别错误/不完整缩写)
            # ==============================================
            "subst", "abus", "prev", "treat", "dept", "univ", "inst", "conf", 
            "proc", "suppl", "nat", "int", "ext", "ci", "or", "rr", "hr", "vs", 
            "etc", "i.e", "e.g", "eg", "ie", "viz", "cf", "et seq", "seq", "ed", 
            "eds", "edn", "rev", "ed rev", "trans", "tr", "vols", "no", "nos", 
            "figs", "tabs", "suppl", "spp", "ms", "msn", "phd", "md", "dr", "prof",
        
            # ==============================================
            # 10. 通用废话修饰词 (无专业价值的形容词/名词)
            # ==============================================
            "great", "good", "bad", "high", "low", "big", "small", "large", 
            "little", "severe", "mild", "moderate", "major", "minor", "primary", 
            "secondary", "tertiary", "main", "chief", "key", "central", "core", 
            "total", "overall", "general", "specific", "particular", "various", 
            "several", "different", "similar", "same", "other", "another", "new", 
            "old", "young", "oldest", "youngest", "positive", "negative", "neutral",
            "common", "rare", "uncommon", "frequent", "infrequent"
        }
        # --- 3. 补充缺失的 bad_starters 属性 (解决 AttributeError 核心) ---
        self.bad_starters = {
            # 无效形容词前缀：以这些词开头的术语直接过滤
            "great", "good", "bad", "high", "low", "big", "small", "large",
            "significant", "total", "mean", "average", "various", "several",
            "different", "similar", "other", "new", "old", "positive", "negative",
            "primary", "secondary", "main", "major", "minor", "severe", "mild",
            "moderate", "general", "specific", "common", "rare", "frequent",
            "infrequent", "little", "chief", "key", "central", "core", "overall",
            "particular", "same", "another", "young", "oldest", "youngest",
            "neutral", "uncommon"
        }

        self.stop_words = set(stopwords.words('english'))

    @staticmethod
    def robust_clean(text):
        if not text: return ""
        text = re.sub(r'(\w+)-\s*\n\s*(\w+)', r'\1\2', text)
        text = re.sub(r'[^a-zA-Z\s]', ' ', text)
        text = re.sub(r'\b(\w+)\s+\1\b', r'\1', text, flags=re.IGNORECASE)
        return re.sub(r'\s+', ' ', text).strip().lower()

    def validate_term(self, term):
        """
        不再只返回 True/False，而是返回 (Bool, Reason)
        """
        words = term.split()
        
        # A. 长度硬限制
        if len(term) < 4: return False, "Length (Too Short)"
        if len(words) > 3: return False, "Length (Too Long)"
        
        # B. 黑名单检查 (最严格)
        for w in words:
            if w in self.blacklist_tokens:
                return False, f"Blacklist Token ({w})"
        
        # C. 形容词前缀检查
        if words[0] in self.bad_starters:
            return False, f"Bad Starter ({words[0]})"
        
        # D. 停用词边界
        if words[0] in self.stop_words: return False, "Stopword Start"
        if words[-1] in self.stop_words: return False, "Stopword End"
        
        # E. 垃圾词检测
        for w in words:
            if len(w) < 2: return False, "Single Char Noise"
            if not re.search(r'[aeiouy]', w): return False, "No Vowels (Garbage)"
            
        return True, "Valid"

    def extract_candidates(self, text):
        # 每次运行前清空日志
        self.rejection_log = []
        
        clean_text = self.robust_clean(text)
        doc = self.nlp(clean_text[:1500000]) 
        matches = self.matcher(doc)
        
        candidates = set()
        # 使用 Counter 记录拒绝原因，防止日志过大
        rejection_stats = []
        
        for _, start, end in matches:
            span = doc[start:end]
            term = span.text.strip()
            
            is_valid, reason = self.validate_term(term)
            
            if is_valid:
                candidates.add(term)
            else:
                # 记录拒绝信息
                self.rejection_log.append({
                    "Rejected Term": term,
                    "Reason": reason
                })
                
        return list(candidates)

    def compute_tfidf(self, pages_list, vocabulary):
        if not vocabulary: return {}
        print("📊 正在计算 TF-IDF 矩阵...")
        vectorizer = TfidfVectorizer(
            vocabulary=vocabulary,
            preprocessor=self.robust_clean,
            ngram_range=(1, 3),
            norm='l2'
        )
        try:
            X = vectorizer.fit_transform(pages_list)
            scores = X.sum(axis=0).A1
            return dict(zip(vectorizer.get_feature_names_out(), scores))
        except Exception as e:
            print(f"⚠️ TF-IDF Error: {e}")
            return {}

# 初始化
extractor = TerminologyExtractor(nlp)
print("✅ 核心算法引擎已构建 (带审计追踪功能)")

✅ 核心算法引擎已构建 (带审计追踪功能)


In [68]:
# === Cell 3.5: 术语排除原因深度分析 (Exclusion Analysis) ===
import pandas as pd

def analyze_rejections(extractor_instance):
    if not extractor_instance.rejection_log:
        print("⚠️ 审计日志为空，请先运行提取流程。")
        return
    
    # 1. 转换为 DataFrame
    df_log = pd.DataFrame(extractor_instance.rejection_log)
    
    # 2. 统计总览
    total_rejected = len(df_log)
    unique_rejected = df_log['Rejected Term'].nunique()
    print(f"🛑 共拦截 {total_rejected} 次，涉及 {unique_rejected} 个唯一短语。")
    print("-" * 60)
    
    # 3. 按原因分类统计
    reason_counts = df_log['Reason'].value_counts().head(10)
    print("📊 拦截原因 Top 10:")
    print(reason_counts)
    print("-" * 60)
    
    # 4. 生成详细样本表 (去重后)
    # 按原因分组，每组取前 3 个例子
    df_unique = df_log.drop_duplicates(subset=['Rejected Term'])
    
    # 展示几个典型的被拦截案例
    print("🔍 典型被排除术语示例 (按原因分类):")
    
    sample_reasons = [
        "Blacklist Token", 
        "Bad Starter", 
        "No Vowels (Garbage)", 
        "Stopword Start"
    ]
    
    report_data = []
    
    for reason_keyword in sample_reasons:
        # 模糊匹配原因 (因为 Blacklist Token 后面有具体词)
        mask = df_unique['Reason'].str.contains(reason_keyword, regex=False)
        samples = df_unique[mask]['Rejected Term'].head(5).tolist()
        
        if samples:
            report_data.append({
                "主要原因类别": reason_keyword,
                "被排除的术语 (Sample)": ", ".join(samples)
            })
    
    display(pd.DataFrame(report_data))
    
    return df_unique

# 运行分析 (确保你已经运行过提取流程 extract_candidates)
# 如果还没运行提取，这里只是定义函数
if 'full_text' in locals() and extractor:
    # 临时运行一次提取以生成日志 (如果你还没跑 Cell 4 的话)
    # 如果已经跑了 Cell 4，直接调用 analyze_rejections(extractor) 即可
    _ = extractor.extract_candidates(full_text) 
    rejected_df = analyze_rejections(extractor)
else:
    print("ℹ️ 请在运行完 pipeline 后调用 analyze_rejections(extractor)")

ℹ️ 请在运行完 pipeline 后调用 analyze_rejections(extractor)


In [69]:
# === Cell 4: 综合评分与 Pipeline 执行 ===
import pandas as pd
from collections import Counter

def run_full_pipeline(full_text, pages_corpus, config):
    print("🚀 Step 1: 正在提取候选术语...")
    # 从全文中提取符合模式的短语
    candidates = extractor.extract_candidates(full_text)
    print(f"   -> 找到 {len(candidates)} 个唯一候选词")
    
    print("🚀 Step 2: 计算 TF-IDF 权重...")
    # 将候选词作为词汇表，去 pages_corpus 中扫描权重
    tfidf_scores = extractor.compute_tfidf(pages_corpus, candidates)
    
    print("🚀 Step 3: 统计频率与评分...")
    # 在清洗后的文本中统计真实频率
    clean_full_text = DocumentProcessor.normalize_text(full_text)
    
    results = []
    
    # 为了效率，我们只对候选词进行迭代
    for term in candidates:
        # 精确匹配频率 (防止 'net' 匹配到 'network')
        # 使用 split() 简单计数作为近似值，比 regex 快得多
        # 为了精确，我们还是用 count，但在清洗过的文本上
        freq = clean_full_text.count(term)
        
        if freq < config['min_freq']: continue
        
        tfidf_val = tfidf_scores.get(term, 0)
        
        # === 评分公式 ===
        # Score = (log(Freq) * w_freq) + (TFIDF * w_tfidf)
        # 我们用简单的线性加权
        
        # 1. 词长奖励 (通常越长的专业术语越具体)
        len_bonus = 1.2 if len(term.split()) > 1 else 1.0
        
        final_score = (freq * config['w_freq'] + tfidf_val * config['w_tfidf']) * len_bonus
        
        # 查找原文语境 (Snippet)
        # 注意：要在原始 full_text (raw) 里找，这样保留大小写和标点
        # 我们需要先大概定位
        raw_lower = full_text.lower()
        start_idx = raw_lower.find(term)
        if start_idx != -1:
            ctx_start = max(0, start_idx - 30)
            ctx_end = min(len(full_text), start_idx + len(term) + 30)
            context = "..." + full_text[ctx_start:ctx_end].replace('\n', ' ') + "..."
        else:
            context = ""
            
        results.append({
            "Term": term,
            "Score": round(final_score, 4),
            "Freq": freq,
            "TF-IDF": round(tfidf_val, 4),
            "Context": context
        })
        
    df = pd.DataFrame(results)
    return df.sort_values("Score", ascending=False)

# =======================
# 🎛️ 权重配置区域 (User Config)
# =======================
config = {
    "min_freq": 3,       # 忽略出现少于 3 次的词
    "w_freq": 0.5,       # 频率权重 (低一些，防止常用词刷屏)
    "w_tfidf": 10.0      # 专业度权重 (拉高，强调“本文特有”的词)
}

# 执行
if 'raw_text' in locals():
    df_final = run_full_pipeline(raw_text, raw_pages, config)
    
    # 显示结果
    print(f"\n📊 最终提取了 {len(df_final)} 个高质量术语")
    display(df_final.head(1500))
else:
    print("⚠️ 请先运行 Cell 2 下载并解析 PDF")

🚀 Step 1: 正在提取候选术语...
   -> 找到 483 个唯一候选词
🚀 Step 2: 计算 TF-IDF 权重...
📊 正在计算 TF-IDF 矩阵...
🚀 Step 3: 统计频率与评分...

📊 最终提取了 77 个高质量术语


,Term,Score,Freq,TF-IDF,Context
35,self attention,34.1650,25,1.5971,
51,attention heads,20.3157,6,1.3930,"...ws (A), we vary the number of attention heads and the attention key and val..."
54,dot product,19.0523,16,0.7877,"... dimension dv. We compute the dot products of the query with all keys, ..."
11,machine translation,17.9175,13,0.8431,... entirely. Experiments on two machine translation tasks show these models to be...
61,sub layer,14.8364,12,0.6364,
67,head attention,13.9286,10,0.6607,...fect we counteract with Multi-Head Attention as described in section 3.2. ...
9,neural networks,13.8803,9,0.7067,...ex recurrent or convolutional neural networks that include an encoder and a...
47,multi head,13.3503,10,0.6125,
71,attention mechanism,12.9066,11,0.5256,"... Transformer, based solely on attention mechanisms, dispensing with recurrence ..."
30,product attention,12.8356,9,0.6196,"...ork. Noam proposed scaled dot-product attention, multi-head attention and the..."


In [ ]:
# === Cell 5: 结果导出 ===
output_file = "Academic_Glossary.xlsx"

if 'df_final' in locals() and not df_final.empty:
    # 格式化导出
    export_df = df_final.rename(columns={
        "Term": "英文术语",
        "Score": "推荐分",
        "Freq": "词频",
        "TF-IDF": "专业度",
        "Context": "原文语境"
    })
    
    # 插入空白翻译列
    export_df.insert(1, "中文翻译", "")
    
    try:
        export_df.to_excel(output_file, index=False)
        print(f"🎉 导出成功！文件位置: {os.path.abspath(output_file)}")
    except Exception as e:
        print(f"❌ 导出失败 (请关闭 Excel 文件后重试): {e}")
else:
    print("⚠️ 没有数据可导出")